# Abstract

Vinyl is a sequential deep learning model (recursive neural network or RNN) that characterizes the genre (or genres) of a song based on its groove, where the groove is defined by the timing, tempo, and musical qualities of the notes played in a song.

I collect data from Spotify for playlists from close to 3,000 musical genres, and obtain audio analysis data for 100-200 songs per genre. By training an RNN a sequence of each note played in each song, I build a model that is able to identify the genres that influence a song. 

Genres are not well defined at a fine level of detail, and this methodology is intended to use the vague boundaries between musical genres to produce a map or genealogy of the evolution of musical styles. This is useful for understanding the history and development of music, and for helping listeners to explore new musical styles that should be similar to their tastes.

# Obtain the Data

To build this model, I pull data from a website called [Every Noise At Once](http://everynoise.com/engenremap.html), which maps out close to 3,000 genres of music in a space that roughly characterized by instrumentation that trends from organic to electric on the vertical axis, anda musical quality that ranges from dense and atmospheric to spiky and bouncy along the horizontal axis.

Each of these genres has its own page that contains a word cloud of popular artists in the genre, as well as links to Spotify playlists with 100-200 songs that represent the genre's style. I scrape Every Noise to collect a list of playlist URIs for each genre, and then I use the Spotify API to collect audio analysis files for the songs in each playlist.

*After completing this step, be sure to edit `references/data_dictionary` to include descriptions of where you obtained your data and what information it contains.*

In [7]:
## %%writefile ../src/data/make_dataset.py

import os, pickle, re, requests
import pandas as pd
from bs4 import BeautifulSoup


def load_or_make(filepath, creator, *args, **kwargs):
    """
    Loads data that is pickled at filepath if filepath exists;
    otherwise, calls creator(*args, **kwargs) to create the data 
    and pickle it at filepath.
    Returns the data in either case.
    
    Inputs:
    - filepath: path to where data is / should be stored
    - creator: function to create data if it is not already pickled
    - *args, **kwargs: arguments passed to creator()
    
    Outputs:
    - item: the data that is stored at filepath
    """
    if os.path.isfile(filepath):
        with open(filepath, 'rb') as pkl:
            item = pickle.load(pkl)
    else:
        item = creator(*args, **kwargs)
        with open(filepath, 'wb') as pkl:
            pickle.dump(item, pkl)
    return item


def scrape_all_links(domain, index, target_pattern):
    """
    Scrapes a website and compiles a list of urls that match a target pattern.
    
    Inputs: 
    - domain: domain of the website you want to scrape
    - index: path to the page that you want to scrape from `domain`
    - target_pattern: regex that specifies the types of links you want to collect
    
    Outputs:
    - target_urls: list of all the links on domain/index that match target_pattern
    """
    main_page = '/'.join(['http:/', domain, index])
    response = requests.get(main_page)

    if response.status_code != 200:
        raise ConnectionError(f"Failed to connect to {main_page}.")

    soup = BeautifulSoup(response.text, "lxml")

    target_regex = re.compile(target_pattern)
    target_urls = ['/'.join(['http:/', domain, x['href']])
                    for x in soup.find_all('a', {'href':target_regex})]

    return target_urls


def scrape_links_from_each_page(urls, target_pattern, labeler=(lambda x:x)):
    """
    Loops over a list of urls and finds links that matches a target pattern from each page.
    
    Inputs:
    - urls: the list of urls to scrape links from
    - target_pattern: regex that specifies the types of links you want to collect
    - labeler: function that parses a url and returns a label for that page
    
    Outputs:
    - links: a dictionary with key/value pairs {url_label:[scraped_links]}
    """
    links = {}

    for url in urls:
        response = requests.get(url)
        label = labeler(url)

        if response.status_code != 200:
            raise ConnectionError(f"Failed to connect to {url}.")

        soup = BeautifulSoup(response.text, "lxml")

        target_regex = re.compile(target_pattern)
        target_urls = [x['href'] for x in soup.find_all('a', {'href':target_regex})]

        links[label] = target_urls
    
    return links


## I DON'T KNOW HOW TO AUTHENTICATE A SPOTIFY CLIENT FROM WITHIN A JUPYTER NOTEBOOK!
## I'LL HAVE TO DO THE NEXT PARTS FROM THE COMMAND LINE.

def download_playlist_metadata(playlists, spotify_client, data_dir):
    """
    Downloads metadata of all songs from each Spotify playlist in playlist_urls,
    and saves them in DataFrames in data_dir.
    
    Inputs:
    - playlists: a dictionary with k/v pairs {playlist_name:playlist_url}
    - spotify_client: a Spotify API client created and authenticated using spotipy
    - data_dir: the directory where metadata DataFrames are to be stored
    
    Outputs:
    - none, but DataFrames are written as TSV files in data_dir
    """
    pass


def download_audio_analysis_files(playlist_df, spotify_client, data_dir):
    """
    Downloads Spotify audio analysis files for all songs in a Spotify playlist
    and pickles them in the data_dir directory. Note that the Spotify client
    must be created and authenticated in the environment outside of this script.

    TODO: find out how to programmatically authenticate a Spotify client object.
    
    Inputs:
    - playlist_df: a DataFrame containing metadata from songs of a Spotify playlist
    - spotify_client: a Spotify API client created and authenticated using spotipy
    - data_dir: the directory where audio analysis data is to be stored
    
    Outputs:
    - none, but audio analysis files are downloaded and pickled in data_dir
    """
    pass

        
def run():
    """
    - scrape genre page urls from everynoise.com/engenremap.html,
        save as a list in ../data/raw/everynoise_genre_urls.pkl
        
    - scrape genre playlist urls from each genre page on everynoise.com,
        save as a dictionary in ../data/raw/thesoundsofspotify_playlist_urls.pkl
        
    - download playlist metadata for each playlist from Spotify,
        save as TSV files in ../data/raw/thesoundsofspotify/[genre].tsv
        
    - download audio_analysis files for each song in a list of playlists
        (not necessarily all playlists because there are 100s of 1000s in the full set)
        save as audio_analysis dictionaries in ../data/raw/audio_analysis/[song_uri].pkl
    
    TODO: include a progress indicator
    """
    genre_urls = load_or_make(
        '../data/raw/everynoise_genre_urls.pkl',
        scrape_all_links, 
        domain='everynoise.com', 
        index='engenremap.html', 
        target_pattern='engenremap-[a-z]*')
    
    genre_playlists = load_or_make(
        '../data/raw/thesoundsofspotify_playlist_urls.pkl',
        scrape_links_from_each_page,
        urls=genre_urls,
        labeler=(lambda url: url.split('/')[-1].split('-')[-1].split('.')[0]),
        target_pattern='open.spotify.com')


run()

In [8]:
genre_urls = load_or_make(
    '../data/raw/everynoise_genre_urls.pkl',
    scrape_all_links, 
    domain='everynoise.com', 
    index='engenremap.html', 
    target_pattern='engenremap-[a-z]*')

genre_playlists = load_or_make(
    '../data/raw/thesoundsofspotify_playlist_urls.pkl',
    scrape_links_from_each_page,
    urls=genre_urls,
    labeler=(lambda url: url.split('/')[-1].split('-')[-1].split('.')[0]),
    target_pattern='open.spotify.com')


In [ ]:
## RUN THIS CODE IN AN IPYTHON SESSION WITH AN AUTHENTICATED SPOTIFY CLIENT
## TODO: FIGURE OUT HOW TO AUTHENTICATE FROM INSIDE A RUNNING PYTHON PROGRAM
## WITHOUT HAVING TO COPY/PASTE URI FROM BROWSER INTO PROMPT

def parse_track_tags(track):
    '''
    Parse metadata for a spotify track, return as a dict
    From a user_playlist json file, a track can be found via:
        user_playlist['tracks']['items'][i]
    '''
    return {
        'id': track['id'],
        'album': track['album']['name'],
        'track': track['track_number'],
        'title': track['name'],
        'artist': track['artists'][0]['name'],
        'duration': int(track['duration_ms']/1000),
        'preview_mp3': track['preview_url'],
        'is_explicit': track['explicit'],
        'isrc_number': track['external_ids']['isrc'],
        'release_date': track['album']['release_date'],
        'cover_art_url': track['album']['images'][0]['url']
    }


def get_playlist_metadata(tracks, client):
    """
    Downloads metadata for each track in tracks, using Spotify API client.
    Downloads audio features for each track and appends to metadata df.
    
    Inputs:
    - tracks: Spotify tracks node from playlist json data
    - client: Spotify API client
    
    Outputs:
    - metadata_df: a DataFrame merged from playlist metadata and 
        audio features downloaded for each song in the playlist
    """
    metadata = [parse_track_tags(item['track']) for item in tracks['items']]
    metadata_df = pd.DataFrame(metadata)
    song_ids = list(metadata_df['id'])
    features_df = pd.DataFrame(client.audio_features(song_ids))
    metadata_df = pd.merge(metadata_df, features_df)

    return metadata_df


def download_tracks_analysis(metadata, client):
    """
    Download audio analysis files for all tracks in playlist, using Spotify API client.
    """
    for index, row in metadata.iterrows():
        filepath = f'data/raw/audio_analysis/{row.id}.pkl'
        if not os.path.isfile(filepath):
            analysis = client.audio_analysis(row.id)
            with open(filepath, 'wb') as pkl:
                pickle.dump(analysis, pkl)

def download_user_playlist_tracks(client, user, playlist_id):
    """
    Calls Spotify client.user_playlist to get a playlist's JSON metadata, 
    and only returns the tracks metadata.
    
    Inputs:
    - client: a spotipy Spotify client object
    - user: Spotify username of the playlist creator
    - playlist_id: the Spotify uri of the playlist
    
    Outputs:
    - tracks
    """
    playlist_json = client.user_playlist(user, playlist_id, fields="tracks,next")
    return playlist_json['tracks']


def get_sos_playlist_id(genre_playlists, genre):
    """
    Parses out the Spotify uri of a genre's playlist created by The Sounds Of Spotify,
    from a list of urls on that genre's subpage on Every Noise At Once.
    
    Inputs:
    - genre_playlists: a dict of format {genre:[playlist_urls]} for every genre on EveryNoise
    - genre: a string, genre name as it appears on EveryNoise (lowercase, hypen-joined)
    
    Outputs:
    - playlist_id: a Spotify uri to that playlist
    """
    sos_playlist = [x for x in genre_playlists[genre] if 'thesoundsofspotify' in x][0]
    return sos_playlist.split('/')[-1]


def run_playlist_metadata_pipeline(genre, genre_playlists, client):
    """
    This is a pipeline to
        get the playlist id for a genre name,
        download playlist tracks,
        download and save playlist metadata,
        download and save tracks audio analysis        
    """
    pl_id = get_sos_playlist_id(genre_playlists, genre)
    tracks = client.user_playlist('thesoundsofspotify', pl_id, fields="tracks,next")['tracks']
    with open(f'data/raw/playlist_json/{genre}.pkl', 'wb') as pkl:
        pickle.dump(tracks, pkl)
    metadata = get_playlist_metadata(tracks, sp)
    metadata.to_csv(f'data/interim/{genre}_metadata.tsv', sep='\t')
    download_tracks_analysis(metadata, sp)

run_playlist_metadata_pipeline('techno', genre_playlists, sp)
run_playlist_metadata_pipeline('opera', genre_playlists, sp)




In [ ]:
genre_tracks = load_or_make(
    f'data/raw/playlist_json/{genre}.pkl',
    download_user_playlist_tracks,
    client = sp,
    user = 'thesoundsofspotify', 
    playlist_id = pl_id)


### Data Dictionary
```
data/
|-raw/everynoise_genre_urls.pkl
| |  /thesoundsofspotify_playlist_urls.pkl
| |--/audio_analysis/{uri}.pkl
| `--/playlist_json/{genre}.pkl
`-interim/{genre}_metadata.tsv

TODO: Write a function that walks the data directory tree and prints this output.
```

Descriptions:
- **everynoise_genre_urls.pkl**: `list` of urls for all genre subpages on everynoise.com
- **thesoundsofspotify_playlist_urls.pkl**: `dict` of genre:urls for all EveryNoise genres; urls point to Spotify playlists by users "The Sounds of Spotify" and "Particle Detector"
- **{uri}.pkl**: `dict` of Audio Analysis JSON file downloaded from Spotify for song {uri}
- **{genre}.pkl**: `dict` of a playlist JSON data file downloaded from Spotify
- **{genre}\_metadata.tsv**: `DataFrame` compiled from track metadata parsed from playlist JSON, merged with a DataFrame of Audio Features data downloaded from Spotify.

# Scrub the Data

*Look through the raw data files and see what you will need to do to them in order to have a workable data set. If your source data is already well-formatted, you may want to ask yourself why it hasn't already been analyzed and what other people may have overlooked when they were working on it. Are there other data sources that might give you more insights on some of the data you have here?*

*The end goal of this step is to produce a [design matrix](https://en.wikipedia.org/wiki/Design_matrix), containing one column for every variable that you are modeling, including a column for the outputs, and one row for every observation in your data set. It needs to be in a format that won't cause any problems as you visualize and model your data.*

In [8]:
## %%writefile ../src/features/build_features.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/raw, cleans them,
    and converts the data into a design matrix that is ready for modeling.
    """
    # clean_dataset_1('data/raw', filename)
    # clean_dataset_2('data/raw', filename)
    # save_cleaned_data_1('data/interim', filename)
    # save_cleaned_data_2('data/interim', filename)
    # build_features()
    # save_features('data/processed')
    pass


*Before moving on to exploratory analysis, write down some notes about challenges encountered while working with this data that might be helpful for anyone else (including yourself) who may work through this later on.*

# Explore the Data

*Before you start exploring the data, write out your thought process about what you're looking for and what you expect to find. Take a minute to confirm that your plan actually makes sense.*

*Calculate summary statistics and plot some charts to give you an idea what types of useful relationships might be in your dataset. Use these insights to go back and download additional data or engineer new features if necessary. Not now though... remember we're still just trying to finish the MVP!*

In [ ]:
## %%writefile ../src/visualization/visualize.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # data = load_features('data/processed')
    # describe_features(data, 'reports/')
    # generate_charts(data, 'reports/figures/')
    pass


*What did you learn? What relationships do you think will be most helpful as you build your model?*

# Model the Data

*Describe the algorithm or algorithms that you plan to use to train with your data. How do these algorithms work? Why are they good choices for this data and problem space?*

In [ ]:
## %%writefile ../src/models/train_model.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # data = load_features('data/processed/')
    # train, test = train_test_split(data)
    # save_train_test(train, test, 'data/processed/')
    # model = build_model()
    # model.fit(train)
    # save_model(model, 'models/')
    pass


In [ ]:
## %%writefile ../src/models/predict_model.py

# imports
# helper functions go here

def run():
    """
    Executes a set of helper functions that read files from data/processed,
    calculates descriptive statistics for the population, and plots charts
    that visualize interesting relationships between features.
    """
    # test_X, test_y = load_test_data('data/processed')
    # trained_model = load_model('models/')
    # predictions = trained_model.predict(test_X)
    # metrics = evaluate(test_y, predictions)
    # save_metrics('reports/')
    pass



_Write down any thoughts you may have about working with these algorithms on this data. What other ideas do you want to try out as you iterate on this pipeline?_

# Interpret the Model

_Write up the things you learned, and how well your model performed. Be sure address the model's strengths and weaknesses. What types of data does it handle well? What types of observations tend to give it a hard time? What future work would you or someone reading this might want to do, building on the lessons learned and tools developed in this project?_